In [12]:
# Import modules
import yfinance as yf # https://pypi.org/project/yfinance/
import pandas as pd
from datetime import datetime, timedelta, date
import time
# from reader import feed
from pandas.tseries.frequencies import to_offset
import numpy as np
# from pandas.tseries.offsets import BDay

In [13]:
#start timer
tic = time.perf_counter()

In [14]:
# Provide list
# tickers = pd.read_csv("Input/ETFs.csv") # final script, ETFs
tickers = pd.read_csv("Input/stock_tickers.csv") # final script, stocks
# tickers = ['AAPL'] # for testing level1
# tickers = ['AAPL','MSFT','AMD'] # for testing level2
# tickers = ['BTC-USD','ETH-USD','LTC-USD'] # for testing level3

In [15]:
# create empty dataframe which the code will append
dfout = pd.DataFrame()

# primary loop for pulling data, doing calculations, and merging dataframes
for ticker in tickers:
    df = yf.Ticker(ticker)
    df = df.history('3y')
#     df.to_csv('Output/cp1.csv')  # data checkpoint1 for data pull
    df.drop(columns=['Dividends','Stock Splits'],inplace=True)
    df1w = yf.download(ticker,period='3y',interval='1wk')
#     df1w = df.history('3y') # not needed
    df1w = df1w.dropna(how='any')
#     df1w.to_csv('Output/cp2.csv')  # data checkpoint2 for data pull
#     df.drop(columns=['Dividends','Stock Splits'],inplace=True) # only needed for df.history('3y') method

#     df3d = yf.download(ticker,period='3y',interval='1d') # used for 3 day calculation check
#     df3d.to_csv('Output/cp3.csv')  # data checkpoint3
    
#     df3d = df.resample('3B').apply(logic) # used for 3day calculation check
#     df3d.index = df3d.index + to_offset('3B') # not needed
#     df3d.to_csv('Output/cp4.csv')  # data checkpoint4
    
# Daily timeframe calculations
    df['conv_line'] = ((df.High.rolling(9).max())+(df.Low.rolling(9).min()))/2 #checked
    df['base_line'] = ((df.High.rolling(26).max())+(df.Low.rolling(26).min()))/2 #checked
    df['lead_a'] = ((df['conv_line']+df['base_line'])/2).shift(25) # checked
    df['lead_b'] = (((df.High.rolling(52).max())+(df.Low.rolling(52).min()))/2).shift(25) # checked
    df['lag_span'] = df.Close.shift(-25) #checked
    df['crit1'] = (df.conv_line>df.base_line).astype(int)  # conversion line above baseline, checked
    df['crit2'] = (df.lead_a.shift(-25)>df.lead_a.shift(-24)).astype(int)  # lead 1 moving up, checked
    df['crit3'] = (df.Close>df.lead_a).astype(int)  # price above Lead 1, checked
    df['crit4'] = (df.Close>df.lead_b).astype(int)  # price above Lead 2, checked
    df['crit5'] = (df.lead_a.shift(-25)>df.lead_b.shift(-25)).astype(int)  # lead 1 above lead 2, checked
    df['crit6'] = (((df.lag_span>df.High).shift(25)).fillna(0).astype(int)) # lag line above price action, checked
    df['crit7'] = (df.lead_b.shift(-25)>df.lead_b.shift(-24)).astype(int)  # lead 2 moving up, checked
    # df['test2'] = df.lead_a.shift(-25)-df.lead_a.shift(-24)  #  this code used to check crit2
    # df['test3'] = df.Close - df.lead_a # used to check crit3
    # df['test4'] = df.Close - df.lead_b # used to check crit4
    # df['test5'] = df.lead_a.shift(-25)-df.lead_b.shift(-25)  # test for crit5
    # df['test6'] = (df.lag_span - df.High).shift(25) # test for crit6
    # df['test7'] = df.lead_b.shift(-25)-df.lead_b.shift(-24)  #  this code used to check crit7
#     df = df.drop(df.columns[[0,1,2,3,4,5,6,7,8,9]],axis=1, inplace=False)  #remove unnecessary columns
#     df.to_csv('Output/cp5.csv')  # data checkpoint5

# 3 day data calculation based on Daily data
# note: this method doesn't line up exactly with TradingView but
#       is needed to merge the daily and 3 day timeframes data
    x = 3 # this number corresponds to the number of days you want to convert daily data into
    df['op'] = df.Open  # checked
    df['hi'] = df.High.rolling(x).max().shift(1-x) # checked, avg dev = -0.1% +/- 0.4%
    df['lo'] = df.Low.rolling(x).min().shift(1-x) # checked, avg dev = 0.1% +/- 0.6%
    df['clse'] = df.Close.shift(1-x) # checked, avg dev = 0.0% +/- 1.1%
    df['vol'] = (df.Volume.shift(1-x) + df.Volume.shift(2-x) + df.Volume) # checked, avg dev = -2.8% +/- 10.0%
    df['conv_line2'] = (((df.hi.rolling(9*x).max())+(df.lo.rolling(9*x).min()))/2).shift(1-x) # checked, avg dev = -2.8% +/- 10.0%
    df['base_line2'] = (((df.hi.rolling(26*x).max())+(df.lo.rolling(26*x).min()))/2).shift(1-x) #checked, avg dev = -0.1% +/- 1.1%
    df['lead_a2'] = ((df['conv_line2']+df['base_line2'])/2).shift(25*x) # checked, avg dev = 0.2% +/- 1.0%
    df['lead_b2'] = (((df.hi.rolling(52*x).max())+(df.lo.rolling(52*x).min()))/2).shift(25*x) # checked, , avg dev = 0.9% +/- 1.5%
    df['lag_span2'] = df.clse.shift(-25*x) #checked, avg dev = -0.3% +/- 4.1%
    df['crit1_3D'] = (df.conv_line2>df.base_line2).astype(int)  # conversion line above baseline, checked
    df['crit2_3D'] = (df.lead_a2.shift(-25*x)>df.lead_a2.shift(-24*x)).astype(int)  # lead 1 moving up, checked
    df['crit3_3D'] = (df.clse>df.lead_a2).astype(int)  # price above Lead 1, checked
    df['crit4_3D'] = (df.clse>df.lead_b2).astype(int)  # price above Lead 2, checked
    df['crit5_3D'] = (df.lead_a2.shift(-25*x)>df.lead_b2.shift(-25*x)).astype(int)  # lead 1 above lead 2, checked
    df['crit6_3D'] = (((df.lag_span2>df.hi).shift(25*x)).fillna(0).astype(int)) # lag line above price action, checked
    df['crit7_3D'] = (df.lead_b2.shift(-25*x)>df.lead_b2.shift(-24*x)).astype(int)  # lead 2 moving up, checked
    df = df.drop(df.columns[[0,1,2,3,4,5,6,7,8,9]],axis=1, inplace=False)  #remove unnecessary columns
    df = df.drop(df.columns[[7,8,9,10,11,12,13,14,15,16]],axis=1, inplace=False)  #remove unnecessary columns
#     df.to_csv('Output/cp6.csv')  # data checkpoint6

# Weekly data calculations
    df1w['conv_line'] = ((df1w.High.rolling(9).max())+(df1w.Low.rolling(9).min()))/2 #checked
    df1w['base_line'] = ((df1w.High.rolling(26).max())+(df1w.Low.rolling(26).min()))/2 #checked
    df1w['lead_a'] = ((df1w['conv_line']+df1w['base_line'])/2).shift(25) # checked
    df1w['lead_b'] = (((df1w.High.rolling(52).max())+(df1w.Low.rolling(52).min()))/2).shift(25) # checked
    df1w['lag_span'] = df1w.Close.shift(-25) #checked
    df1w['crit1'] = (df1w.conv_line>df1w.base_line).astype(int)  # conversion line above baseline, checked
    df1w['crit2'] = (df1w.lead_a.shift(-25)>df1w.lead_a.shift(-24)).astype(int)  # lead 1 moving up, checked
    df1w['crit3'] = (df1w.Close>df1w.lead_a).astype(int)  # price above Lead 1, checked
    df1w['crit4'] = (df1w.Close>df1w.lead_b).astype(int)  # price above Lead 2, checked
    df1w['crit5'] = (df1w.lead_a.shift(-25)>df1w.lead_b.shift(-25)).astype(int)  # lead 1 above lead 2, checked
    df1w['crit6'] = (((df1w.lag_span>df1w.High).shift(25)).fillna(0).astype(int)) # lag line above price action, checked
    df1w['crit7'] = (df1w.lead_b.shift(-25)>df1w.lead_b.shift(-24)).astype(int)  # lead 2 moving up, checked
    # df1w['test2'] = df1w.lead_a.shift(-25)-df1w.lead_a.shift(-24)  #  this code used to check crit2
    # df1w['test3'] = df1w.Close - df1w.lead_a # used to check crit3
    # df1w['test4'] = df1w.Close - df1w.lead_b # used to check crit4
    # df1w['test5'] = df1w.lead_a.shift(-25)-df1w.lead_b.shift(-25)  # test for crit5
    # df1w['test6'] = (df1w.lag_span - df1w.High).shift(25) # test for crit6
    # df1w['test7'] = df1w.lead_b.shift(-25)-df1w.lead_b.shift(-24)  #  this code used to check crit7
    df1w = df1w.drop(df1w.columns[[0,1,2,3,4,5,6,7,8,9,10]],axis=1, inplace=False)  #remove unnecessary columns
#     df1w.to_csv('Output/cp7.csv')  # data checkpoint7

# Reset indexes for proper merging
    df.reset_index(inplace=True)
    df1w.reset_index(inplace=True)
#     df1w.to_csv('Output/cp8.csv')  # data checkpoint8

# Merge daily, 3day and weekly data
    merged_df = pd.merge(df,\
                         df1w,\
                         left_on=[df['Date'].dt.year, df['Date'].dt.strftime('%U')],\
                         right_on=[df1w['Date'].dt.year, df1w['Date'].dt.strftime('%U')],\
                         how='right',suffixes=('_1D','_1W'),).drop(['key_0','key_1'],axis=1)
    merged_df=merged_df.apply(lambda x: x.fillna(method='ffill') if x.dtype.kind in \
                              'biufc' else x.fillna(method='ffill')) # fill in blank data with previous row data
    merged_df = merged_df.drop_duplicates(subset=['Date_1D'],keep='first') # remove duplicate values
#     merged_df.to_csv('Output/cp9.csv')  # data checkpoint9

# Formatting merged data fraomes
# move daily date data to front of dataframe
    merged_df = merged_df.drop(merged_df.columns[[15]],axis=1, inplace=False)  #remove weekly date column
    col_list = list(merged_df) # remove Date column to prevent Type Error during summing
    col_list.remove('Date_1D')
    merged_df['total_pts'] = merged_df[col_list].sum(axis=1)  # adds up the points, checked good
#     merged_df.to_csv('Output/cp10.csv')  # data checkpoint10
    merged_df = merged_df.drop(merged_df.columns[[1,2,3,4,5,6,7,8,9,10,\
                             11,12,13,14,15,16,17,\
                             18,19,20,21]],axis=1,inplace=False) # remove unnecessary columns, checked good
    row_count = len(merged_df.index) # get the row count of the dataframe
    merged_df = merged_df.iloc[row_count-2: , :] # remove all rows except the last two
    chg = merged_df.total_pts.diff() # find difference between the last two rows
    merged_df['chg'] = chg # add the chg data to the dataframe
    row_count = len(merged_df.index) # get the NEW row count of the dataframe
    merged_df = merged_df.iloc[row_count-1: , :] # remove all but the last row
    merged_df['ticker'] = ticker # add ticker label column
    merged_df = merged_df[['Date_1D','ticker','total_pts','chg']] # move ticker column to the front
    dfout = dfout.append(merged_df)

dfout = dfout.sort_values(['total_pts','chg'],ascending=(False,False))
# indout = indout.sort_values(['sector','industry','z_score_total'],ascending=(True,True,False))
dfout.to_csv('Output/Ichimoku_Output.csv')
# print(ticker)
# pd.set_option('display.max_rows',999)
dfout.head(10)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


<ipython-input-15-89efdb6cf5dc>:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1w['conv_line'] = ((df1w.High.rolling(9).max())+(df1w.Low.rolling(9).min()))/2 #checked
<ipython-input-15-89efdb6cf5dc>:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1w['base_line'] = ((df1w.High.rolling(26).max())+(df1w.Low.rolling(26).min()))/2 #checked
<ipython-input-15-89efdb6cf5dc>:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


<ipython-input-15-89efdb6cf5dc>:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1w['conv_line'] = ((df1w.High.rolling(9).max())+(df1w.Low.rolling(9).min()))/2 #checked
<ipython-input-15-89efdb6cf5dc>:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1w['base_line'] = ((df1w.High.rolling(26).max())+(df1w.Low.rolling(26).min()))/2 #checked
<ipython-input-15-89efdb6cf5dc>:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


...

<ipython-input-15-89efdb6cf5dc>:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1w['conv_line'] = ((df1w.High.rolling(9).max())+(df1w.Low.rolling(9).min()))/2 #checked
<ipython-input-15-89efdb6cf5dc>:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1w['base_line'] = ((df1w.High.rolling(26).max())+(df1w.Low.rolling(26).min()))/2 #checked
<ipython-input-15-89efdb6cf5dc>:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


<ipython-input-15-89efdb6cf5dc>:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1w['conv_line'] = ((df1w.High.rolling(9).max())+(df1w.Low.rolling(9).min()))/2 #checked
<ipython-input-15-89efdb6cf5dc>:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1w['base_line'] = ((df1w.High.rolling(26).max())+(df1w.Low.rolling(26).min()))/2 #checked
<ipython-input-15-89efdb6cf5dc>:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

[*********************100%***********************]  1 of 1 completed


,Date_1D,ticker,total_pts,chg
749,2021-09-07,OPGN,8.0,3.0
749,2021-09-07,MGY,8.0,0.0
749,2021-09-07,GOOG,8.0,0.0
749,2021-09-07,GOOGL,8.0,0.0
749,2021-09-07,MTL,8.0,0.0
173,2021-09-07,FCUV,8.0,0.0
749,2021-09-07,MSFT,8.0,0.0
749,2021-09-07,ANY,8.0,0.0
749,2021-09-07,BBIG,8.0,0.0
717,2021-09-07,GAZ,8.0,0.0


In [16]:
merged_df.dtypes

Date_1D      datetime64[ns]
ticker               object
total_pts             int64
chg                 float64
dtype: object

In [17]:
# Reference code

# alternative method for use with pulling yfinance data
# today = date.today()
# print("Today's date:", today)
# days_back = 365*3
# rows_drop = days_back-2
# start_date = (date.today() - timedelta(days=days_back)).strftime('%Y-%m-%d')
# start_date
# df = yf.download(tickers,start= start_date, end=date.today())
# df.head()

# logic for resampling method code
# logic = {'Open'  : 'first',
#          'High'  : 'max',
#          'Low'   : 'min',
#          'Close' : 'last',
#          'Volume': 'sum'}

# this section is accurate for 3day timeframe and used to verify results of calculated 3 day timeframe
#     df3d['conv_line'] = ((df3d.High.rolling(9).max())+(df3d.Low.rolling(9).min()))/2 #checked
#     df3d['base_line'] = ((df3d.High.rolling(26).max())+(df3d.Low.rolling(26).min()))/2 #checked
#     df3d['lead_a'] = ((df3d['conv_line']+df3d['base_line'])/2).shift(25) # checked
#     df3d['lead_b'] = (((df3d.High.rolling(52).max())+(df3d.Low.rolling(52).min()))/2).shift(25) # checked
#     df3d['lag_span'] = df3d.Close.shift(-25) #checked
#     df3d['crit1'] = (df3d.conv_line>df3d.base_line).astype(int)  # conversion line above baseline, checked
#     df3d['crit2'] = (df3d.lead_a.shift(-25)>df3d.lead_a.shift(-24)).astype(int)  # lead 1 moving up, checked
#     df3d['crit3'] = (df3d.Close>df3d.lead_a).astype(int)  # price above Lead 1, checked
#     df3d['crit4'] = (df3d.Close>df3d.lead_b).astype(int)  # price above Lead 2, checked
#     df3d['crit5'] = (df3d.lead_a.shift(-25)>df3d.lead_b.shift(-25)).astype(int)  # lead 1 above lead 2, checked
#     df3d['crit6'] = (((df3d.lag_span>df3d.High).shift(25)).fillna(0).astype(int)) # lag line above price action, checked
#     df3d['crit7'] = (df3d.lead_b.shift(-25)>df3d.lead_b.shift(-24)).astype(int)  # lead 2 moving up, checked
    # df3d['test2'] = df3d.lead_a.shift(-25)-df3d.lead_a.shift(-24)  #  this code used to check crit2
    # df3d['test3'] = df3d.Close - df3d.lead_a # used to check crit3
    # df3d['test4'] = df3d.Close - df3d.lead_b # used to check crit4
    # df3d['test5'] = df3d.lead_a.shift(-25)-df3d.lead_b.shift(-25)  # test for crit5
    # df3d['test6'] = (df3d.lag_span - df3d.High).shift(25) # test for crit6
    # df3d['test7'] = df3d.lead_b.shift(-25)-df3d.lead_b.shift(-24)  #  this code used to check crit7
#     df3d.to_csv('Output/df3d.csv')  # data checkpoint
#     df3d = df3d.drop(df3d.columns[[0,1,2,3,4,5,6,7,8,9]],axis=1, inplace=False)  #remove unnecessary columns
#     df3d.to_csv('Output/df3d.csv')  # data checkpoint

In [18]:
toc = time.perf_counter()
print(f"Total processing time is {toc-tic:0.4f} seconds")

Total processing time is 119.3595 seconds
